# Data Cleaning 

In [416]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [103]:
#rename columns
def rename_col(df, col_name,new_name):
    '''
    Renames column in a dataframe
    Arguments:
        df: data frame
        col_name: the column name to be renamed
        new_name: new column name
        
    Returns:
        None
        
    '''
    df.rename({col_name:new_name}, axis=1, inplace=True)
    

### Rename some columns from all three data sets.

In [421]:
# rename id column of portfolio dataframe
rename_col(portfolio,  'id', 'offer_id')

In [422]:
# rename id column of profile dataframe
rename_col(profile,  'id', 'customer_id')

In [423]:
# rename person column of transcript dataframe
rename_col(transcript,  'person', 'customer_id')

## I) Clean portfolio data

### Column 'offer_type' contains categorical data which can be converted to one hot encoding.

In [424]:
#pd.get_dummies(portfolio['offer_type'])
portfolio= portfolio.join(pd.get_dummies(portfolio['offer_type']))
#portfolio

In [425]:
# Remove 'offer_type' column
portfolio = portfolio.drop('offer_type', 1)

### Channel column contains categorical data with different categories. We create a separate column for each category.

In [426]:
# Clean channels colums, create distinct columns from channel columns
channels_df = pd.DataFrame(portfolio['channels'])
channels_df = channels_df.explode('channels')
channels_df = channels_df.assign(value=lambda x: 1)
channels_df = channels_df.pivot(columns='channels', values='value')
channels_df.fillna(value=0, inplace=True)
portfolio = portfolio.join(channels_df)
channels_df = None


In [427]:
# Drop 'channels' column
portfolio = portfolio.drop('channels', 1)

In [428]:
# View cleaned portfolio data 
portfolio

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1.0,1.0,1.0,0.0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1.0,1.0,1.0,1.0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1.0,1.0,0.0,1.0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1.0,1.0,0.0,1.0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1.0,0.0,0.0,1.0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1.0,1.0,1.0,1.0
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1.0,1.0,1.0,1.0
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1.0,1.0,1.0,0.0
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1.0,1.0,1.0,1.0
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1.0,1.0,0.0,1.0


### Email channel is used by all offer types, so we can drop the email column.

In [429]:
portfolio = portfolio.drop('email', 1)

In [430]:
portfolio

,reward,difficulty,duration,offer_id,bogo,discount,informational,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1.0,1.0,0.0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1.0,1.0,1.0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1.0,0.0,1.0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1.0,0.0,1.0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,0.0,0.0,1.0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1.0,1.0,1.0
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1.0,1.0,1.0
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1.0,1.0,0.0
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1.0,1.0,1.0
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1.0,0.0,1.0


## II) Clean profile data.

In [431]:
profile.head()

,gender,age,customer_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [432]:
profile.shape

(17000, 5)

In [433]:
#Check nan values
profile.isnull().sum()

gender              2175
age                    0
customer_id            0
became_member_on       0
income              2175
dtype: int64

###  Gender and income values are missing. The gender column consists of distinct categorical data, the misisng gender can be assigned as a separate category 'N'.

In [434]:
#Number of rows with nan 
profile.isnull().any(axis=1).sum()

2175

In [435]:
# Missing gender values
gender_isna=profile['gender'].isna()

In [436]:
# Create a new 'N' category for misisng not reported gender
profile['gender'].mask(gender_isna, 'N', inplace=True)

In [437]:
# Make one hot encoded vector for 4 gender categories
profile= profile.join(pd.get_dummies(profile['gender']))

In [438]:
#drop gender
profile= profile.drop('gender', 1)

In [439]:
#Check the data
profile

,age,customer_id,became_member_on,income,F,M,N,O
0,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,0,0,1,0
1,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0,0
2,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,0,0,1,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0,0
4,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...
16995,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,0,0
16996,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,0,1,0,0
16997,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,0,1,0,0
16998,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,0,0,0


### The income column comprises of continuous variables; therefore, the missing values can be imputed by mean.

In [440]:
profile['income'] = profile['income'].fillna(profile['income'].mean())

In [441]:
profile['age'].isna().sum()

0

### The became_member_on is on a string format, let's transform it to the long form. 

In [442]:
profile['became_member_on']=pd.to_datetime(profile['became_member_on'], format='%Y%m%d').astype(np.long)

In [443]:
profile['became_member_on'].isnull().sum()

0

## III) Clean Transcript data

In [445]:
transcript

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


### The 'value' column is split into four columns: 'offer id', 'amount', 'offer_id' and 'reward'.

In [446]:
transcript = transcript.join(
    pd.DataFrame.from_records(transcript['value']))

In [447]:
transcript=transcript.drop('value',1)

In [448]:
transcript

,customer_id,event,time,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,NaN,1.59,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,NaN,9.53,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,NaN,3.61,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,NaN,3.53,NaN,NaN


### We can see two columns for offer id ('offer id' and 'offer_id'). From the data analysis section, we figured out that offer_id contains ids for offer completed only. We can merger the two columns into one.

In [450]:
transcript.offer_id.update(transcript['offer id'])

In [451]:
transcript=transcript.drop('offer id',1)

In [453]:
transcript

,customer_id,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,NaN


In [462]:
# Check how many person received offer
group = transcript.groupby('event')
df2 = group.apply(lambda x: x['customer_id'].unique())
print('Total unique users received offer:{}'.format(len(df2['offer received'])))

Total unique users received offer:16994


### The offer was sent to almost everyone (16994 out of 17000). So, let's filter one who received offer at first.

In [467]:
offer_received_df = transcript.query('event == "offer received"')

In [468]:
# Check the offer sending times
offer_sending_time = offer_received_df.time.unique()
print('Offer sent time:', offer_sending_time)

Offer sent time: [  0 168 336 408 504 576]


In [470]:
## Set customer_id and time
offer_received_df = offer_received_df.set_index(['customer_id', 'time'])
offer_received_df = offer_received_df.sort_index()

In [471]:
offer_received_df

event  amount  \
customer_id                      time                           
0009655768c64bdeb2e877511632db8f 168   offer received     NaN   
                                 336   offer received     NaN   
                                 408   offer received     NaN   
                                 504   offer received     NaN   
                                 576   offer received     NaN   
...                                               ...     ...   
ffff82501cea40309d5fdd7edcca4a07 168   offer received     NaN   
                                 336   offer received     NaN   
                                 408   offer received     NaN   
                                 504   offer received     NaN   
                                 576   offer received     NaN   

                                                               offer_id  \
customer_id                      time                                     
0009655768c64bdeb2e877511632db8f 168   5a8bc65990b245e5a138643cd4eb9837   
                                 336   3f207df678b143eea3cee63160fa8bed   
                                 408   f19421c1d4aa40978ebb69ca19b0e20d   
                                 504   fafdcd668e3743c1bb461111dcafc2a4   
                                 576   2906b810c7d4411798c6938adc9daaa5   
...                                                                 ...   
ffff82501cea40309d5fdd7edcca4a07 168   0b1e1539f2cc45b7b9fa7c272da2e1d7   
                                 336   2906b810c7d4411798c6938adc9daaa5   
                                 408   2906b810c7d4411798c6938adc9daaa5   
                                 504   9b98b8c7a33c4b65b9aebfe6a799e6d9   
                                 576   2906b810c7d4411798c6938adc9daaa5   

                                       reward  
customer_id                      time          
0009655768c64bdeb2e877511632db8f 168      NaN  
                                 336      NaN  
                                 408      NaN  
                                 504      NaN  
                                 576      NaN  
...                                       ...  
ffff82501cea40309d5fdd7edcca4a07 168      NaN  
                                 336      NaN  
                                 408      NaN  
                                 504      NaN  
                                 576      NaN  

[76277 rows x 4 columns]

In [475]:
# Set offer_id as index in portfolio.
portfolio=portfolio.set_index(['offer_id'])

In [474]:
portfolio

,reward,difficulty,duration,bogo,discount,informational,mobile,social,web
offer_id,,,,,,,,,
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,1,0,0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,1,0,0,1.0,1.0,1.0
3f207df678b143eea3cee63160fa8bed,0,0,4,0,0,1,1.0,0.0,1.0
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,1,0,0,1.0,0.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,0,1,0,0.0,0.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,0,1,0,1.0,1.0,1.0
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,0,1,0,1.0,1.0,1.0
5a8bc65990b245e5a138643cd4eb9837,0,0,3,0,0,1,1.0,1.0,0.0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,0,0,1.0,1.0,1.0


In [476]:
# Add duration in hours column from portfolio to offer_received dataframe 
offer_received_df= offer_received_df.join(portfolio.duration*24, on='offer_id')

In [477]:
# Add inoformational column from portfolio to offer_received df
offer_received_df= offer_received_df.join(portfolio_new.informational, on='offer_id')

In [478]:
offer_received_df

event  amount  \
customer_id                      time                           
0009655768c64bdeb2e877511632db8f 168   offer received     NaN   
                                 336   offer received     NaN   
                                 408   offer received     NaN   
                                 504   offer received     NaN   
                                 576   offer received     NaN   
...                                               ...     ...   
ffff82501cea40309d5fdd7edcca4a07 168   offer received     NaN   
                                 336   offer received     NaN   
                                 408   offer received     NaN   
                                 504   offer received     NaN   
                                 576   offer received     NaN   

                                                               offer_id  \
customer_id                      time                                     
0009655768c64bdeb2e877511632db8f 168   5a8bc65990b245e5a138643cd4eb9837   
                                 336   3f207df678b143eea3cee63160fa8bed   
                                 408   f19421c1d4aa40978ebb69ca19b0e20d   
                                 504   fafdcd668e3743c1bb461111dcafc2a4   
                                 576   2906b810c7d4411798c6938adc9daaa5   
...                                                                 ...   
ffff82501cea40309d5fdd7edcca4a07 168   0b1e1539f2cc45b7b9fa7c272da2e1d7   
                                 336   2906b810c7d4411798c6938adc9daaa5   
                                 408   2906b810c7d4411798c6938adc9daaa5   
                                 504   9b98b8c7a33c4b65b9aebfe6a799e6d9   
                                 576   2906b810c7d4411798c6938adc9daaa5   

                                       reward  duration  informational  
customer_id                      time                                   
0009655768c64bdeb2e877511632db8f 168      NaN        72              1  
                                 336      NaN        96              1  
                                 408      NaN       120              0  
                                 504      NaN       240              0  
                                 576      NaN       168              0  
...                                       ...       ...            ...  
ffff82501cea40309d5fdd7edcca4a07 168      NaN       240              0  
                                 336      NaN       168              0  
                                 408      NaN       168              0  
                                 504      NaN       168              0  
                                 576      NaN       168              0  

[76277 rows x 6 columns]

### Drop amount and reward column as they contain only NaNs.

In [481]:
offer_received_df=offer_received_df.drop(columns=['amount', 'reward'])

In [482]:
offer_received_df

event  \
customer_id                      time                   
0009655768c64bdeb2e877511632db8f 168   offer received   
                                 336   offer received   
                                 408   offer received   
                                 504   offer received   
                                 576   offer received   
...                                               ...   
ffff82501cea40309d5fdd7edcca4a07 168   offer received   
                                 336   offer received   
                                 408   offer received   
                                 504   offer received   
                                 576   offer received   

                                                               offer_id  \
customer_id                      time                                     
0009655768c64bdeb2e877511632db8f 168   5a8bc65990b245e5a138643cd4eb9837   
                                 336   3f207df678b143eea3cee63160fa8bed   
                                 408   f19421c1d4aa40978ebb69ca19b0e20d   
                                 504   fafdcd668e3743c1bb461111dcafc2a4   
                                 576   2906b810c7d4411798c6938adc9daaa5   
...                                                                 ...   
ffff82501cea40309d5fdd7edcca4a07 168   0b1e1539f2cc45b7b9fa7c272da2e1d7   
                                 336   2906b810c7d4411798c6938adc9daaa5   
                                 408   2906b810c7d4411798c6938adc9daaa5   
                                 504   9b98b8c7a33c4b65b9aebfe6a799e6d9   
                                 576   2906b810c7d4411798c6938adc9daaa5   

                                       duration  informational  
customer_id                      time                           
0009655768c64bdeb2e877511632db8f 168         72              1  
                                 336         96              1  
                                 408        120              0  
                                 504        240              0  
                                 576        168              0  
...                                         ...            ...  
ffff82501cea40309d5fdd7edcca4a07 168        240              0  
                                 336        168              0  
                                 408        168              0  
                                 504        168              0  
                                 576        168              0  

[76277 rows x 4 columns]

In [484]:
#Reset index
offer_received_df=offer_received_df.reset_index()

In [485]:
offer_received_df

,customer_id,time,event,offer_id,duration,informational
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,72,1
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,96,1
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,120,0
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,240,0
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,168,0
...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,240,0
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,168,0
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,168,0
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,0


In [509]:
#Create extra columns
## Create column to indicate when the offer ends
offer_received_df = offer_received_df.assign(
    offer_ends_on=offer_received_df.time + offer_received_df.duration)

## Create column to indicate when the offer is viewed or completed
offer_received_df = offer_received_df.assign(viewed_on=np.nan, completed_on=np.nan)

## Create a column to hold the label for that offer sending
offer_received_df = offer_received_df.assign(label=np.nan)

In [510]:
offer_received_df

,customer_id,time,event,offer_id,duration,informational,offer_ends_on,viewed_on,completed_on,label
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,72,1,240,NaN,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,96,1,432,NaN,NaN,NaN
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,120,0,528,NaN,NaN,NaN
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,240,0,744,NaN,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,744,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,240,0,408,NaN,NaN,NaN
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,504,NaN,NaN,NaN
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,576,NaN,NaN,NaN
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,0,672,NaN,NaN,NaN


In [511]:
offer_viewed_df = transcript.query('event == "offer viewed"')
offer_completed_df = transcript.query('event == "offer completed"')
transaction_df = transcript.query('event == "transaction"')

In [686]:
print('Number of unique customers who received offer:',len(offer_received_df['customer_id'].unique()))
print('Number of unique customers who viewed offer:',len(offer_viewed_df['customer_id'].unique()))
print('Number of unique customers who completed offer:',len(offer_completed_df['customer_id'].unique()))
print('Number of unique customers who did transactions:',len(transaction_df['customer_id'].unique()))

Number of unique customers who received offer: 16994
Number of unique customers who viewed offer: 16834
Number of unique customers who completed offer: 12774
Number of unique customers who did transactions: 16578


### Create 3 types of labels.
### 1)  No transctions: label 0
### 2) Transactions under the influence of offer: label 1
### 3) Transaction without offer : leabel 2

In [766]:
def assign_labels(row):
    '''
    Create levels for each row in the offer_received_df.
    Filters offer by compares the time between the offer received, offer viewed, offer expires and trasaction.
    uses splitted dataframes: offer_viewed_df, offer_completed_df and tranaction_df.
    
    Arguments:
        dataframe row
    Returns:
        row (with labels)
    
    '''
    '''
    this segment assigns offer viewed time to each customer_id and offer_id, if not viewed, label=0
    '''
    try:
        '''
        >For a given customer_id and offer_id on each row, 
          if the offer viewed time is within offer received and  offer expires time, that offer is considered 'viewed'
        > assigned the time to viewed_on column
        '''
        row.viewed_on = offer_viewed_df.query(
            'customer_id == @row.customer_id ' \
            'and offer_id == @row.offer_id ' \
            'and time >= @row.time ' \
            'and time <= @row.offer_ends_on').time.values[0]
        
    except:
        '''if offer not viewed, try offer completed without viewing the offer''' 
        try:
            dummy = offer_completed_df.query(
            'customer_id == @row.customer_id ' \
            'and offer_id == @row.offer_id ' \
            'and time >= @row.time ' \
            'and time <= @row.offer_ends_on').time.values[0]
            #row.label= 1
            #return row
            
        except:
            '''offer not viewed and not completed, but still doing the transaction.
               This section is the transaction without the infulence of offer
            '''
            try:
                dummy = transaction_df.query(
                'customer_id == @row.customer_id ' \
                'and time >= @row.time ' \
                'and time <= @row.offer_ends_on').time.values[0]
                #row.label= 2
                #return row
            except:
                '''label 0 for no transaction'''
                row.label = 0
                #return row
            else:
                '''transaction without using offer'''
                row.label= 2
            #finally:
             #   return row
        else:
            '''transaction or offer completed without viewing it'''
            row.label= 1
        finally:
            return row
        

    '''this segment checks the rows for which the offer are infiormational'''
    if row.informational == 1:
        try:
            '''Informational offer are viewed not completed 
               Transaction completed during the time interval when the informational offer are sent 
               is assigned to completed_on column
            '''
            row.completed_on = transaction_df.query(
                'customer_id == @row.customer_id ' \
                'and time >= @row.viewed_on ' \
                'and time <= @row.offer_ends_on ').time.values[0]
        except:
            ''' there is no transaction in this period'''
            row.label = 0
        else:
            row.label = 2
        finally:
            return row

    ''' this segment checks for offers viewed and offer completed'''
    try:
        '''check for offer completion and assign time on completed_on column'''
        row.completed_on = offer_completed_df.query(
            'customer_id == @row.customer_id ' \
            'and offer_id == @row.offer_id ' \
            'and time >= @row.viewed_on ' \
            'and time <= @row.offer_ends_on').time.values[0]
    except:
        ''' the offer was not completed '''
        row.label = 0
    else:
        row.label = 1
    finally:
        return row

In [753]:
df_test= offer_received_df.sample(1000)

In [754]:
df_test


,customer_id,time,event,offer_id,duration,informational,offer_ends_on,viewed_on,completed_on,label
8776,1ebff9c4dddc439e864de60c6fe01fb6,336,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,168,0,504,NaN,NaN,NaN
22840,4c96ee37b5bb4d389177ca83222d5d84,336,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,0,504,NaN,NaN,NaN
70674,ecd3d9913d5b48d2bd60aa7f73fee1cc,0,offer received,3f207df678b143eea3cee63160fa8bed,96,1,96,NaN,NaN,NaN
33268,6fc4b2ee5d844566aafeaa209fe11de1,0,offer received,ae264e3637204a6fb9bb56bc8210ddfd,168,0,168,NaN,NaN,NaN
65368,dadae228edc442d59471e7e8d42e1d9e,576,offer received,fafdcd668e3743c1bb461111dcafc2a4,240,0,816,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7569,1a55c37ef53944cda20b4f98f11ef0d8,336,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,240,0,576,NaN,NaN,NaN
25080,54570dbe1e5a43ec8036fff68d1a1a10,168,offer received,ae264e3637204a6fb9bb56bc8210ddfd,168,0,336,NaN,NaN,NaN
51862,ac7651d3f0f24f789ad9b2c16451bafd,168,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,336,NaN,NaN,NaN
1017,03887756866f4ea9b74b7e1b83ffac44,504,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,672,NaN,NaN,NaN


In [759]:
df_test = df_test.apply(assign_labels_2, axis=1)


In [760]:
len(df_test[df_test['label']==0])

413

In [761]:
len(df_test[df_test['label']==1])

398

In [762]:
len(df_test[df_test['label']==2])

189

In [767]:
offer_received_df

,customer_id,time,event,offer_id,duration,informational,offer_ends_on,viewed_on,completed_on,label
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,72,1,240,NaN,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,96,1,432,NaN,NaN,NaN
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,120,0,528,NaN,NaN,NaN
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,240,0,744,NaN,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,744,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,240,0,408,NaN,NaN,NaN
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,504,NaN,NaN,NaN
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,168,0,576,NaN,NaN,NaN
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,0,672,NaN,NaN,NaN


In [768]:
offer_received_df = offer_received_df.apply(assign_labels, axis=1)

In [770]:
len(offer_received_df[offer_received_df['label']==0])

32795

In [771]:
len(offer_received_df[offer_received_df['label']==1])

29598

In [772]:
len(offer_received_df[offer_received_df['label']==2])

13884

In [776]:
# save to pickle
offer_received_df.to_pickle('offer_received_with_labels.pkl')
#load pickle
#offer_received_df_reload = pd.read_pickle('offer_received_with_labels.pkl')

In [779]:
## Remove auxiliary columns
offer_received_df.drop(
    inplace=True,
    columns=['duration', 'offer_ends_on', 'informational',
             'viewed_on', 'completed_on'])

In [780]:
offer_received_df

,customer_id,time,event,offer_id,label
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,2
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,2
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,0
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,1
...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,1
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,1
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0


In [783]:
# merge dataframes offer_received and portfolio
offer_received_df=offer_received_df.join(portfolio, on='offer_id')

In [789]:
#Change customer_id as profile index
profile=profile.set_index(['customer_id'])
profile

,age,became_member_on,income,F,M,N,O
customer_id,,,,,,,
68be06ca386d4c31939f3a4f0e3dd783,118,1486857600000000000,65404.991568,0,0,1,0
0610b486422d4921ae7d2bf64640c50b,55,1500076800000000000,112000.000000,1,0,0,0
38fe809add3b4fcf9315a9694bb96ff5,118,1531353600000000000,65404.991568,0,0,1,0
78afa995795e4d85b5d9ceeca43f5fef,75,1494288000000000000,100000.000000,1,0,0,0
a03223e636434f42ac4c3df47e8bac43,118,1501804800000000000,65404.991568,0,0,1,0
...,...,...,...,...,...,...,...
6d5f3a774f3d4714ab0c092238f3a1d7,45,1528070400000000000,54000.000000,1,0,0,0
2cb4f97358b841b9a9773a7aa05a9d77,61,1531440000000000000,72000.000000,0,1,0,0
01d26f638c274aa0b965d24cefe3183f,49,1485388800000000000,73000.000000,0,1,0,0


In [791]:
# Final merged dataframe
merged_df=offer_received_df.join(profile, on='customer_id')

In [792]:
merged_df

,customer_id,time,event,offer_id,label,reward,difficulty,duration,bogo,discount,...,mobile,social,web,age,became_member_on,income,F,M,N,O
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,2,0,0,3,0,0,...,1.0,1.0,0.0,33,1492732800000000000,72000.0,0,1,0,0
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,4,0,0,...,1.0,0.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,5,5,5,1,0,...,1.0,1.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,2,10,10,0,1,...,1.0,1.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,7,0,1,...,1.0,0.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,20,10,0,1,...,0.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,7,0,1,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,7,0,1,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5,5,7,1,0,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0


In [806]:
merged_df['duration']=merged_df['duration']*24

In [808]:
rename_col(merged_df, 'duration','offer_duration_hrs')

In [809]:
merged_df

,customer_id,time,event,offer_id,label,reward,difficulty,offer_duration_hrs,bogo,discount,...,mobile,social,web,age,became_member_on,income,F,M,N,O
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,2,0,0,72,0,0,...,1.0,1.0,0.0,33,1492732800000000000,72000.0,0,1,0,0
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,1.0,0.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,5,5,120,1,0,...,1.0,1.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,2,10,240,0,1,...,1.0,1.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,168,0,1,...,1.0,0.0,1.0,33,1492732800000000000,72000.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,20,240,0,1,...,0.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76273,ffff82501cea40309d5fdd7edcca4a07,336,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,168,0,1,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76274,ffff82501cea40309d5fdd7edcca4a07,408,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,168,0,1,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0
76275,ffff82501cea40309d5fdd7edcca4a07,504,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5,5,168,1,0,...,1.0,0.0,1.0,45,1480032000000000000,62000.0,1,0,0,0


### Let's final preprocess 'income' and 'age' columns. 

In [816]:
print('The minimum income is {} and maximum income is {}.'.format(np.min(merged_df['income']), np.max(merged_df['income'])))

The minimum income is 30000.0 and maximum income is 120000.0.


### Divide income into three groups:
###    1) Less than 60000 label as 1
###    2) Between 60000 and 90000 label as 2
###    3) Above 90000 label as 3

In [821]:
def create_income_group(df):
    """
    Creates income group
        
    Arguments:
        df: data frame
    
    Returns:
        df: output dataframe with income_group column
    """
    #add age_group column
    df.loc[(df['income'] < 60000) , 'income_group'] = 1
    df.loc[(df['income'] >= 60000) & (df['income'] < 90000) , 'income_group'] = 2
    df.loc[(df['income'] >= 90000) , 'income_group'] = 3
    return df

In [822]:
merged_df= create_income_group(merged_df)

In [823]:
merged_df.sample(10)

,customer_id,time,event,offer_id,label,reward,difficulty,offer_duration_hrs,bogo,discount,...,social,web,age,became_member_on,income,F,M,N,O,income_group
62275,d04b09c1ae754602aa52f3edad0660e2,168,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,3,7,168,0,1,...,1.0,1.0,118,1517616000000000000,65404.991568,0,0,1,0,2.0
60004,c87e041cfaac44f9a2e3b4791b915ee6,0,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,0.0,1.0,48,1502841600000000000,56000.000000,0,1,0,0,1.0
15360,347b0731981245c681867568552acf2c,408,offer received,2906b810c7d4411798c6938adc9daaa5,0,2,10,168,0,1,...,0.0,1.0,118,1483228800000000000,65404.991568,0,0,1,0,2.0
15353,347a75e83b70480ba7b53c7f737ebb4a,168,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,0.0,1.0,82,1451001600000000000,55000.000000,1,0,0,0,1.0
18219,3d43c355a4bb4d5e8d86a80a87408925,408,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,20,240,0,1,...,0.0,1.0,22,1474848000000000000,36000.000000,1,0,0,0,1.0
60757,cb4a0508e95e41b8a57c59d69a20f06d,576,offer received,f19421c1d4aa40978ebb69ca19b0e20d,1,5,5,120,1,0,...,1.0,1.0,60,1455667200000000000,78000.000000,1,0,0,0,2.0
5163,119f5c7a948046198be6173e377d4bc3,336,offer received,fafdcd668e3743c1bb461111dcafc2a4,1,2,10,240,0,1,...,1.0,1.0,64,1463616000000000000,60000.000000,1,0,0,0,2.0
6312,15a95216e2cd4a14b45fc182adb81dc6,504,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,20,240,0,1,...,0.0,1.0,86,1454976000000000000,92000.000000,0,1,0,0,3.0
61784,ce8b82092e674393ba1c9a03f30d4551,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,1,2,10,240,0,1,...,1.0,1.0,118,1483660800000000000,65404.991568,0,0,1,0,2.0
76132,ff8ff8e5d8bb449894bf708924020edc,576,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,0.0,1.0,78,1465344000000000000,58000.000000,1,0,0,0,1.0


### Divide age into three groups:
###     1) Less than 40 years label as 1
###     2) Between 40 to 80 years label as 2
###     3) Above 80 years label as 3

In [824]:
print('The minimum income is {} and maximum income is {}.'.format(np.min(merged_df['age']), np.max(merged_df['age'])))

The minimum income is 18 and maximum income is 118.


In [825]:
def create_age_group(df):
    """
    Creates age group column
        
    Arguments:
        df: dataframe
    
    Returns:
        df: output dataframe with age_group column
    """
    #add age_group column
    df.loc[(df['age'] < 40) , 'age_group'] = 1
    df.loc[(df['age'] >= 40) & (df['age'] < 80) , 'age_group'] = 2
    df.loc[(df['age'] >= 80) , 'age_group'] = 3
    return df

In [826]:
merged_df= create_age_group(merged_df)

In [829]:
merged_df.sample(10)

,customer_id,time,event,offer_id,label,reward,difficulty,offer_duration_hrs,bogo,discount,...,web,age,became_member_on,income,F,M,N,O,income_group,age_group
3590,0c3e283397d74cd9a3f535c7dd1d188c,336,offer received,f19421c1d4aa40978ebb69ca19b0e20d,1,5,5,120,1,0,...,1.0,51,1492732800000000000,113000.000000,1,0,0,0,3.0,2.0
5828,13f997b9e6384b809bfe26cb6fa9f8af,408,offer received,5a8bc65990b245e5a138643cd4eb9837,0,0,0,72,0,0,...,0.0,67,1511568000000000000,71000.000000,0,1,0,0,2.0,2.0
53438,b18ede11af8c44cfa56289fd58f04573,336,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,5,5,168,1,0,...,1.0,32,1422057600000000000,37000.000000,0,1,0,0,1.0,1.0
34023,7248fc98eb6c4b1fb0f1a2cec6ba2f93,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,2,10,240,0,1,...,1.0,34,1493942400000000000,65000.000000,0,1,0,0,2.0,1.0
20142,4399fd7521de4731838ac1215975a5d8,504,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,2,5,20,240,0,1,...,1.0,65,1453420800000000000,48000.000000,0,1,0,0,1.0,2.0
68269,e4989ca3d8974e28a3fe87aa0b0ce327,576,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,5,5,168,1,0,...,1.0,46,1532563200000000000,74000.000000,0,1,0,0,2.0,2.0
69968,ea5f76f81dc8488a955172fb91d06bca,504,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,1.0,118,1485129600000000000,65404.991568,0,0,1,0,2.0,3.0
65376,daeccc82b26348ddbffcc3a775b7465f,408,offer received,3f207df678b143eea3cee63160fa8bed,0,0,0,96,0,0,...,1.0,118,1512604800000000000,65404.991568,0,0,1,0,2.0,3.0
23076,4d70dd97b0004d47ad520d33da69b6da,168,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,20,240,0,1,...,1.0,82,1389139200000000000,60000.000000,0,1,0,0,2.0,3.0
16424,37dfdf86ca2044d4a4e0a2eb2904c12c,504,offer received,5a8bc65990b245e5a138643cd4eb9837,2,0,0,72,0,0,...,0.0,72,1519689600000000000,65000.000000,0,1,0,0,2.0,2.0


### Add columns for  offer type and gender for final data analysis.

In [881]:
portfolio_org = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_org = pd.read_json('data/profile.json', orient='records', lines=True)

In [882]:
merged_df.drop(inplace=True, columns=['gender', 'offer_type'])

In [883]:
portfolio_org.rename({'id':'offer_id'}, axis=1, inplace=True)
portfolio_org=portfolio_org.set_index(['offer_id'])
merged_df= merged_df.join(portfolio_org['offer_type'], on='offer_id')

In [884]:
profile_org.rename({'id':'customer_id'}, axis=1, inplace=True)
profile_org=profile_org.set_index(['customer_id'])
#Changing gender category None to N
profile_org['gender'].mask(gender_isna, 'N',inplace=True)
merged_df=merged_df.join(profile_org['gender'], on='customer_id')                           

In [885]:
gender_isna=profile_org['gender'].isna()
profile_org['gender'].mask(gender_isna, 'N', inplace=True)

In [901]:
merged_df.head()

,customer_id,time,event,offer_id,label,reward,difficulty,offer_duration_hrs,bogo,discount,...,became_member_on,income,F,M,N,O,income_group,age_group,offer_type,gender
0,0009655768c64bdeb2e877511632db8f,168,offer received,5a8bc65990b245e5a138643cd4eb9837,2,0,0,72,0,0,...,1492732800000000000,72000.0,0,1,0,0,2.0,1.0,informational,M
1,0009655768c64bdeb2e877511632db8f,336,offer received,3f207df678b143eea3cee63160fa8bed,2,0,0,96,0,0,...,1492732800000000000,72000.0,0,1,0,0,2.0,1.0,informational,M
2,0009655768c64bdeb2e877511632db8f,408,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,5,5,120,1,0,...,1492732800000000000,72000.0,0,1,0,0,2.0,1.0,bogo,M
3,0009655768c64bdeb2e877511632db8f,504,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,2,10,240,0,1,...,1492732800000000000,72000.0,0,1,0,0,2.0,1.0,discount,M
4,0009655768c64bdeb2e877511632db8f,576,offer received,2906b810c7d4411798c6938adc9daaa5,1,2,10,168,0,1,...,1492732800000000000,72000.0,0,1,0,0,2.0,1.0,discount,M


### Save the final merged data frame for exploratory data analysis and model fitting

In [886]:
# save to pickle
merged_df.to_pickle('merged_data.pkl')

## To explore the demographic distribution of offers, create a new dataframe with event and gender columns.

In [887]:
profile_org

,gender,age,became_member_on,income
customer_id,,,,
68be06ca386d4c31939f3a4f0e3dd783,N,118,20170212,NaN
0610b486422d4921ae7d2bf64640c50b,F,55,20170715,112000.0
38fe809add3b4fcf9315a9694bb96ff5,N,118,20180712,NaN
78afa995795e4d85b5d9ceeca43f5fef,F,75,20170509,100000.0
a03223e636434f42ac4c3df47e8bac43,N,118,20170804,NaN
...,...,...,...,...
6d5f3a774f3d4714ab0c092238f3a1d7,F,45,20180604,54000.0
2cb4f97358b841b9a9773a7aa05a9d77,M,61,20180713,72000.0
01d26f638c274aa0b965d24cefe3183f,M,49,20170126,73000.0


In [888]:
transcript_org = pd.read_json('data/transcript.json', orient='records', lines=True)

In [889]:
transcript_org.sample(10)

,person,event,value,time
223063,7047452b0e9344e7b7d7e2e63162ac6b,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},516
213712,f17d13f232ad4eaa973185c5a07d991d,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},504
299626,6b3d6732ed1b4acd8dd1e0b3512d42d9,transaction,{'amount': 17.91},684
238562,2fc548b045ae4cc0baec39e7e3aa5999,transaction,{'amount': 10.52},552
303418,1c780ef3de0d4397ae1d854771813a66,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,702
178437,507e271f7d2f43458ba0162db5c41b3f,transaction,{'amount': 1.8900000000000001},432
132226,390e1e6505e64e64836f27d502522059,transaction,{'amount': 0.30000000000000004},348
295639,29074c62ec28408e94c2bfe22e6e2c37,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,666
29915,4f47639209234af293ca702bf4eaaac3,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},48
260904,747f706b685144b29bbc8a0f209432ec,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},576


In [891]:
#transcript_org.rename({'person':'customer_id'}, axis=1, inplace=True)
transcript_org=transcript_org.set_index(['customer_id'])

In [899]:
transcript_org

,event,value,time
customer_id,,,
78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...
b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714


In [894]:
event_gender_df=transcript_org.join(profile_org['gender'], on='customer_id') 

In [896]:
event_gender_df.drop(inplace=True, columns=['value','time'])

In [897]:
event_gender_df

,event,gender
customer_id,,
78afa995795e4d85b5d9ceeca43f5fef,offer received,F
a03223e636434f42ac4c3df47e8bac43,offer received,N
e2127556f4f64592b11af22de27a7932,offer received,M
8ec6ce2a7e7949b1bf142def7d0e0586,offer received,N
68617ca6246f4fbc85e91a2a49552598,offer received,N
...,...,...
b3a1272bc9904337b331bf348c3e8c17,transaction,M
68213b08d99a4ae1b0dcb72aebd9aa35,transaction,M
a00058cf10334a308c68e7631c529907,transaction,F


### Save the event_gender dataframe

In [898]:
# save to pickle
event_gender_df.to_pickle('event_gender_data.pkl')